In [ ]:
!cp -r ../input/yolomodel/* .

In [ ]:
from utils.datasets import *
from utils.utils import *
from models.experimental import *
import sys
sys.path.insert(0, "../input/weightedboxesfusion")
from ensemble_boxes import *
import torch
import glob
import pandas as pd

In [ ]:
weights = ["../input/40efolds/best48fold0.pt", "../input/40efolds/best48fold1.pt", "../input/40efolds/best48fold2.pt",  "../input/40efolds/best48fold3.pt", "../input/40efolds/best40fold4.pt"]
print(len(weights))
source = "../input/global-wheat-detection/test"
imgsz = 1024
built_in_tta = True

device = torch_utils.select_device("0" if torch.cuda.is_available() else "")
#If using CUDA, we can use half-floats
half = device.type != 'cpu'  # half precision only supported on CUDA

In [ ]:
skip_box_thr = 0.2
iou_thr = 0.57 #current best 0.55<0.6
score_thr = 0.2
def run_wbf(boxes,scores, image_size=1024, iou_thr=0.4, skip_box_thr=0.34, weights=None):
    labels0 = [np.ones(len(scores[idx])) for idx in range(len(scores))]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels0, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    return boxes, scores

In [ ]:
def TTAImage(image, index):
    image1 = image.copy()
    if index==0:  #rotate once
        rotated_image = cv2.rotate(image1, cv2.ROTATE_90_CLOCKWISE)
        return rotated_image
    elif index==1: #rotate twice
        rotated_image2 = cv2.rotate(image1, cv2.ROTATE_90_CLOCKWISE)
        rotated_image2 = cv2.rotate(rotated_image2, cv2.ROTATE_90_CLOCKWISE)
        return rotated_image2
    elif index==2:#rotate thrice
        rotated_image3 = cv2.rotate(image1, cv2.ROTATE_90_CLOCKWISE)
        rotated_image3 = cv2.rotate(rotated_image3, cv2.ROTATE_90_CLOCKWISE)
        rotated_image3 = cv2.rotate(rotated_image3, cv2.ROTATE_90_CLOCKWISE)
        return rotated_image3
    elif index == 3: # four rotatins = original image
        return image1
    
def rotBoxes90(boxes, im_w, im_h):
    ret_boxes =[]
    for box in boxes:
        x1, y1, x2, y2 = box
        x1, y1, x2, y2 = x1-im_w//2, im_h//2 - y1, x2-im_w//2, im_h//2 - y2
        x1, y1, x2, y2 = y1, -x1, y2, -x2
        x1, y1, x2, y2 = int(x1+im_w//2), int(im_h//2 - y1), int(x2+im_w//2), int(im_h//2 - y2)
        x1a, y1a, x2a, y2a = min(x1, x2), min(y1, y2), max(x1, x2), max(y1, y2)
        ret_boxes.append([x1a, y1a, x2a, y2a])
    return np.array(ret_boxes)

def detect1Image(img, img0, model, device, aug):
    #normalize image values
    img = img.transpose(2,0,1)
    img = torch.from_numpy(img).to(device)
    img = img.half() if half else img.float()  # uint8 to fp16/32
    img /= 255.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)
    
    # the prediction(Using Yolo implementation)
    pred = model(img, augment=aug)[0]
    
    # Apply NMS
    pred = non_max_suppression(pred, score_thr, iou_thr, merge=True, classes=None, agnostic=False)
    
    boxes = []
    scores = []
    for i, det in enumerate(pred):  # for every detection
        if det is not None and len(det): 
            # Rescale boxes from img_size to img0 size
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()

            # Store results
            for *xyxy, conf, cls in det:
                boxes.append([int(xyxy[0]), int(xyxy[1]), int(xyxy[2]), int(xyxy[3])])
                scores.append(conf)

    return np.array(boxes), np.array(scores) 

In [ ]:
def detect():
    models = []

    # Load all models using weight.pt files
    for w in weights if isinstance(weights, list) else [weights]:
        models.append(torch.load(w, map_location=device)['model'].to(device).float().eval())
        
    # Load test image dataset using Yolo implementation
    dataset = LoadImages(source, img_size=imgsz)
    
    # init img
    img = torch.zeros((1, 3, imgsz, imgsz), device=device)  
    
    #If using CUDA, turn all models to half float
    for model in models:
        if half:
            model.half()  # to FP16
        _ = model(img.half() if half else img) if device.type != 'cpu' else None  # run once

    all_paths, all_bboxes, all_confs = [], [], []
    for p, img, img0, _ in dataset:
        print()
        img = img.transpose(1,2,0) # [H, W, 3]

        all_paths.append(p)
        m_box, m_score = [], []
        for model in models:
            enboxes = []
            enscores = []
            
            #Detect the image rotated once, twice, three times and not at all
            for i in range(4):
                img1 = TTAImage(img, i)
                boxes, scores = detect1Image(img1, img0, model, device, aug=False)
                for _ in range(3-i):
                    boxes = rotBoxes90(boxes, *img.shape[:2])            
                enboxes.append(boxes)
                enscores.append(scores)
            #Detect the image flipped(based on Yolo implement)
            boxes, scores = detect1Image(img, img0, model, device, aug=True)
            enboxes.append(boxes)
            enscores.append(scores) 
            
            #for the 5 TTA detections , fuse boxes
            boxes, scores = run_wbf(enboxes, enscores, image_size=1024, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
            boxes = boxes.astype(np.int32).clip(min=0, max=1024)
            
            #Ignore low percentage predictions
            indices = scores >= score_thr
            boxes = boxes[indices]
            scores = scores[indices]
            #For each model, store the 'good' boxes and scores
            m_box.append(boxes)
            m_score.append(scores)
            
        all_bboxes.append(m_box)
        all_confs.append(m_score)
    return all_paths, all_bboxes, all_confs

In [ ]:
with torch.no_grad():
    res = detect()
paths,all_boxes,all_confs = res

In [ ]:
results =[]
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))
    return " ".join(pred_strings)
bts = []
sts = []
for row in range(len(paths)):
    image_id = paths[row].split("/")[-1].split(".")[0]
    #Combine ensemble predictions
    boxes, scores = run_wbf(all_boxes[row],all_confs[row], skip_box_thr = skip_box_thr, iou_thr = iou_thr)
    
    #Ignore bad predictions
    indices = scores >= score_thr
    boxes = boxes[indices]
    scores = scores[indices]
    
    #Fix boxes to image size and store them
    boxes = (boxes*1024/1024).astype(np.int32).clip(min=0, max=1023)
    boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
    boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
    result = {'image_id': image_id,'PredictionString': format_prediction_string(boxes, scores)}
    results.append(result)
    
    #boxes to show
    bts.append(boxes)
    #scores to show
    sts.append(scores)
!rm -rf *
test_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
test_df.to_csv('submission.csv', index=False)
test_df.head()

In [ ]:
amount_to_show = 10
shown = []
fig = plt.figure(figsize=[20,20])
per_row = 3
count = amount_to_show
rows = 1

font = cv2.FONT_HERSHEY_SIMPLEX 
fontScale = 1
# color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) 
thickness = 3
while count > per_row:
    count -= per_row
    rows +=1
color = (255, 255, 255)    
for i in range(1, amount_to_show+1):
    fig.add_subplot(rows, per_row, i)
    idx = random.randint(0, 9)
    while idx  in shown:
        idx = random.randint(0, 9)
    shown.append(idx)
    image = image = cv2.imread(paths[idx], cv2.IMREAD_COLOR)
    for b,s in zip(bts[idx],sts[idx]):
        image = cv2.rectangle(image, (b[0],b[1]), (b[0]+b[2],b[1]+b[3]), color, 2) 
        image = cv2.putText(image, '{}{:.2}'.format("Wheat", s), (b[0]+np.random.randint(3),b[1]), font,  
                       fontScale, color, thickness, cv2.LINE_AA)
    
    plt.title(paths[idx].split("/")[-1].split(".")[0])
    plt.imshow(image[:,:,::-1])
    cv2.imwrite(paths[idx].split("/")[-1].split(".")[0] + ".jpg", image)
fig.tight_layout()
plt.show()